In [97]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [143]:
options = Options()
options.add_argument('window-size=1100,800')

navegador = webdriver.Chrome('C:/workspace/scrapy_ifood/chromedriver.exe', options=options)
navegador.get('https://www.ifood.com.br/restaurantes?sort=distance%3Aasc')

In [145]:
while True:
    try:
        sleep(4)
        if navegador.find_element_by_xpath('//*[@id="__next"]/div[1]/main/div/div[2]/section/article[2]/section[2]/div/button'):
            navegador.find_element_by_xpath('//*[@id="__next"]/div[1]/main/div/div[2]/section/article[2]/section[2]/div/button').click()
        else:
            sleep(6)
            if navegador.find_element_by_xpath('//*[@id="__next"]/div[1]/main/div/div[2]/section/article[2]/section[2]/div/button'):
                navegador.find_element_by_xpath('//*[@id="__next"]/div[1]/main/div/div[2]/section/article[2]/section[2]/div/button').click()        
    except:
        break

In [163]:
page_content = navegador.page_source

site = BeautifulSoup(page_content, 'html.parser')

merchants = site.findAll('div', {'class': 'merchant-list-v2__item-wrapper'})

list_merchants = []

In [164]:
for merchant in merchants:
    nome = merchant.find('span', {'class': 'merchant-v2__name'}).get_text()
    enderecoweb = ''.join(['https://www.ifood.com.br']+[merchant.find('a', {'class': 'merchant-v2__link'}).get('href')])
    cidade = enderecoweb.split('/')[4]
    separatexto = merchant.find('div', {'class': 'merchant-v2__info'}).get_text().split(' • ')
    if len(separatexto) >= 3:
        estrelas = merchant.find('div', {'class': 'merchant-v2__info'}).get_text().split(' • ')[0]
    else:
        estrelas = '-'
    segmento = merchant.find('div', {'class': 'merchant-v2__info'}).get_text().split(' • ')[-2]
    distancia = merchant.find('div', {'class': 'merchant-v2__info'}).get_text().split(' • ')[-1]
    tempo = merchant.find('div', {'class': 'merchant-v2__footer'}).get_text().split(' • ')[0]
    frete = merchant.find('div', {'class': 'merchant-v2__footer'}).get_text().split(' • ')[1]
    if merchant.find('span', {'class': 'icon-marmita icon-marmita--star-super'}):
        campea = 'Sim'
    else:
        campea = 'Não'
    list_merchants.append([nome,campea,enderecoweb,cidade,estrelas,segmento,distancia,tempo,frete])

In [165]:
dados = pd.DataFrame(list_merchants, columns=['NomeLoja','LojaCampea','EnderecoWeb','Cidade','Estrelas','Segmento','Distancia','Tempo','Frete'])
dados.to_csv('lojas_ifood.csv', index=False, sep=';')

In [166]:
dados

,NomeLoja,LojaCampea,EnderecoWeb,Cidade,Estrelas,Segmento,Distancia,Tempo,Frete
0,Mania de Açaí,Sim,https://www.ifood.com.br/delivery/igarassu-pe/...,igarassu-pe,4.9,Açaí,0.2 km,30-40 min,"R$ 5,00"
1,Monster Burguer e Pizzaria,Sim,https://www.ifood.com.br/delivery/igarassu-pe/...,igarassu-pe,4.5,Lanches,0.3 km,60-70 min,"R$ 5,00"
2,Pan Josina,Não,https://www.ifood.com.br/delivery/igarassu-pe/...,igarassu-pe,4.8,Padaria,0.4 km,38-48 min,"R$ 4,99"
3,Coxinhas de Batata Igarassu,Não,https://www.ifood.com.br/delivery/igarassu-pe/...,igarassu-pe,4.8,Lanches,0.5 km,35-45 min,"R$ 5,00"
4,Dog 28,Sim,https://www.ifood.com.br/delivery/igarassu-pe/...,igarassu-pe,4.8,Lanches,0.5 km,30-40 min,"R$ 4,00"
...,...,...,...,...,...,...,...,...,...
435,Doces da Lua,Não,https://www.ifood.com.br/delivery/paulista-pe/...,paulista-pe,Novidade,Doces & Bolos,8.2 km,Fechado,"R$ 22,99"
436,Comedoria Paratibe,Não,https://www.ifood.com.br/delivery/paulista-pe/...,paulista-pe,Novidade,Lanches,8.2 km,Fechado,"R$ 22,99"
437,Hamburgueria das Praias,Não,https://www.ifood.com.br/delivery/paulista-pe/...,paulista-pe,-,Hambúrguer,8.2 km,Fechado,"R$ 22,99"
438,Doceria da Vó Zinha,Não,https://www.ifood.com.br/delivery/paulista-pe/...,paulista-pe,5.0,Doces & Bolos,8.3 km,Fechado,"R$ 22,99"
